In [2]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [3]:
table26 = pd.read_excel('Table 26.xlsx')
ratio_length_depth = pd.read_excel('Length depth ratio.xlsx')
table27 = pd.read_excel('Table 27.xlsx')
table19 = pd.read_excel('Table 19.xlsx')
clause40211 = pd.read_excel('clause 40_2_1_1.xlsx')
clause26_2_1_1 = pd.read_excel('clause 26_2_1_1.xlsx')


In [4]:
length_room = float(input("enter the length of the room(in m): "))*1000
breadth_room = float(input("enter the breadth of the room(in m): "))*1000

# determining whether length or breadth is bigger and making breadth bigger
if(length_room>breadth_room):
    temp = length_room
    length_room = breadth_room 
    breadth_room = temp


width_column = float(input("enter the width of column(in mm): "))
diameter_rod = float(input("diameter of rod you want to use in mm: "))
floor_finish = float(input("enter the floor finish pressure(in KN/m*m): "))
live_load = float(input("enter the live load to be considered(in KN/m*m): "))
fck = float(input("enter the fck value(in N/mm*mm) (20, 25, 30, 35, 40): "))
fy = float(input("enter the fy value(250, 415, 500): "))
case = int(input("enter the case number in table length depth ratio(1-9): "))-1


# finding out depth of concrete required by first calculating depth and adding diameter of rod and 15mm of 

depth_concrete_initial = (length_room / ratio_length_depth.iloc[case,1])
print(f"\n\nInitial depth we get is {depth_concrete_initial} mm")

enter the length of the room(in m): 4
enter the breadth of the room(in m): 5
enter the width of column(in mm): 230
diameter of rod you want to use in mm: 10
enter the floor finish pressure(in KN/m*m): 1.2
enter the live load to be considered(in KN/m*m): 4
enter the fck value(in N/mm*mm) (20, 25, 30, 35, 40): 20
enter the fy value(250, 415, 500): 415
enter the case number in table length depth ratio(1-9): 2


Initial depth we get is 125.0 mm


In [5]:
# defining fuction to generate values with the help of table26
def information(a):
    if(a<=1.5):
        index = int(np.floor((a-1)*10))
        alpha_x_negative = table26.iloc[2*case + 1,index+2] + ((table26.iloc[2*case +1,index+3] - table26.iloc[2*case + 1,index+2])/0.1)*(a - (1 + (index/10)))
        alpha_x_positive = table26.iloc[2*case + 2, index+2] + ((table26.iloc[2*case + 2,index+3] - table26.iloc[2*case + 2,index+2])/0.1)*(a - (1 + (index/10)))
        alpha_y_negative = table26.iloc[2*case + 1,10]
        alpha_y_positive = table26.iloc[2*case + 2,10]
    elif(a>1.5 and a<1.75):
        index = 7
        alpha_x_negative = table26.iloc[2*case + 1,index+2] + ((table26.iloc[2*case +1,index+3] - table26.iloc[2*case + 1,index+2])/0.25)*(a - (1.5))
        alpha_x_positive = table26.iloc[2*case + 2, index+2] + ((table26.iloc[2*case + 2,index+3] - table26.iloc[2*case + 2,index+2])/0.25)*(a - (1.5))
        alpha_y_negative = table26.iloc[2*case + 1,10]
        alpha_y_positive = table26.iloc[2*case + 2,10]
    else:
        index = 8
        alpha_x_negative = table26.iloc[2*case + 1,index+2] + ((table26.iloc[2*case +1,index+3] - table26.iloc[2*case + 1,index+2])/0.25)*(a - (1.75))
        alpha_x_positive = table26.iloc[2*case + 2, index+2] + ((table26.iloc[2*case + 2,index+3] - table26.iloc[2*case + 2,index+2])/0.25)*(a - (1.75))
        alpha_y_negative = table26.iloc[2*case + 1,10]
        alpha_y_positive = table26.iloc[2*case + 2,10]
    alpha_max = max(alpha_x_positive,alpha_x_negative,alpha_y_positive,alpha_y_negative)
    return (alpha_max,alpha_x_positive,alpha_x_negative,alpha_y_positive,alpha_y_negative)


In [6]:
def main(depth_concrete_initial):
    if((length_room + depth_concrete_initial) > (length_room + width_column) or (breadth_room + depth_concrete_initial - diameter_rod)>(breadth_room + width_column)):
        print("\n\n\nWarning!!!!!!,You need to increase column width\n\n\n")
    else:
        #calculating depth with mortar and rod
        depth_concrete = depth_concrete_initial + 15 + (diameter_rod/2)
    
        #calcultaing total load and factorial load which would be helpful further

        total_load = floor_finish * 1 + live_load * 1 + 25*1*(depth_concrete/1000) # considering 1 meter of slab and 25 is concrete
        # gamma coefficient
        factorial_load = 1.5 * total_load #1.5 is safety factor

        #calcultaing ratio to generate values for table using defined function

        length_ex = length_room + depth_concrete_initial
        breadth_ey = breadth_room + depth_concrete_initial - diameter_rod
        a = (breadth_ey)/(length_ex)

        if(a<2):
            #storing value of aplha
            alpha_max,alpha_x_positive,alpha_x_negative,alpha_y_positive,alpha_y_negative = information(a)

            # calculating moment and checking value of depth we got

            moment_x_positive = alpha_x_positive * factorial_load * (length_ex**2)
            moment_x_negative = alpha_x_negative * factorial_load * (length_ex**2)
            moment_y_positive = alpha_y_positive * factorial_load * (length_ex**2)
            moment_y_negative = alpha_y_negative * factorial_load * (length_ex**2)
            moment_max = alpha_max * factorial_load * (length_ex**2)

            if(fy==250):
                x_u = 0.53 
            elif(fy==415):
                x_u = 0.48 
            else:
                x_u = 0.46
                
            times_depth_change = 0    
            depth_temp = np.sqrt(moment_max/(0.36*fck*1000*(x_u*(1-0.42*x_u))))
            if(depth_temp > depth_concrete_initial):
                depth_concrete_initial = depth_temp
                times_depth_change +=1
                main(depth_concrete_initial)

            #calculation of Ast

            Ast_minimum = (.12/100)*(1000)*(depth_concrete)
            Ast_x_positive = max(((0.87*fy*depth_concrete_initial)- np.sqrt(((0.87*fy*depth_concrete_initial)**2) - ((4*moment_x_positive*0.87*fy*fy)/(fck*1000))))/((2*0.87*fy*fy)/(fck*1000)),Ast_minimum)
            Ast_x_negative = max(((0.87*fy*depth_concrete_initial)- np.sqrt(((0.87*fy*depth_concrete_initial)**2) - ((4*moment_x_negative*0.87*fy*fy)/(fck*1000))))/((2*0.87*fy*fy)/(fck*1000)),Ast_minimum)
            Ast_y_positive = max(((0.87*fy*depth_concrete_initial)- np.sqrt(((0.87*fy*depth_concrete_initial)**2) - ((4*moment_y_positive*0.87*fy*fy)/(fck*1000))))/((2*0.87*fy*fy)/(fck*1000)),Ast_minimum)
            Ast_y_negative = max(((0.87*fy*depth_concrete_initial)- np.sqrt(((0.87*fy*depth_concrete_initial)**2) - ((4*moment_y_negative*0.87*fy*fy)/(fck*1000))))/((2*0.87*fy*fy)/(fck*1000)),Ast_minimum)
            Ast_individual = (np.pi) * ((diameter_rod / 2)**2)

            #calculating spacing
            if(3*depth_concrete_initial > 300):
                spacing_minimum = 3*depth_concrete_initial
            else:
                spacing_minimum = 300
            
            spacing_x = min((1000)/(Ast_x_negative/Ast_individual),spacing_minimum)
            spacing_x_midspan = min((1000)/(Ast_x_positive/Ast_individual),spacing_minimum)
            spacing_y = min((1000)/(Ast_y_negative/Ast_individual),spacing_minimum)
            spacing_y_midspan = min((1000)/(Ast_y_positive/Ast_individual),spacing_minimum)
            
            
            #shear load
            v_max =( factorial_load * (length_ex/1000))/2
            torque_max = (factorial_load * length_ex)/(2*(depth_concrete_initial*1000))
            
            #calculating k from clause 40_2_1_1
            if(depth_concrete_initial <= 150):
                k=1.3
            elif(depth_concrete_initial >=300):
                k = 1
            else:
                k = 1.3 + ((depth_concrete_initial - 150)*(-0.3/150))
            
            def Ast_shear_function(spacing):
                Ast_shear = (1000*Ast_individual)/spacing
                temp_shear = Ast_shear/(10*depth_concrete_initial)
                
                #indices as matrix(x,y)
                
                if(( fck - 10)/5 <= 6):
                    concrete_index_y = int(( fck - 10)/5)
                else:
                    concrete_index_y = 6
                if(temp_shear<= 0.15):
                    concrete_index_x = 1
                    torque_c = .28
                elif(temp_shear >= 3):
                    concrete_index_x = 13
                    torque_c = .71
                else:
                    concrete_index_x = int((np.floor(temp_shear*100))//25) + (1)
                    torque_c = (((temp_shear - table19.iloc[concrete_index_x,0])*(table19.iloc[concrete_index_x,concrete_index_y]-table19.iloc[concrete_index_x+ 1,concrete_index_y ]))/(table19.iloc[concrete_index_x,0]-table19.iloc[concrete_index_x+ 1,0 ])) + table19.iloc[concrete_index_x,concrete_index_y]
                torque_perm = k * torque_c
                return torque_perm, temp_shear
            
            temp_index_midspan = 0
            temp_index_spacing = 0
            if(Ast_shear_function(spacing_x_midspan)[0]<torque_max):
                while(Ast_shear_function(spacing_x_midspan)[0]<torque_max):
                    spacing_x_midspan = spacing_x_midspan - 1
                    temp_index_midspan +=1
                    if(temp_index_midspan>7):
                        break
                Ast_x_positive = (1000*Ast_individual)/spacing_x_midspan
                
                if(Ast_shear_function(spacing_x)[0]<torque_max):
                    while(Ast_shear_function(spacing_x)[0]<torque_max):
                        spacing_x = spacing_x -1
                        temp_index_spacing += 1
                        if(temp_index_spacing>7):
                            break
                    Ast_x_negative=(1000*Ast_individual)/spacing_x
            
            
            #defelection check
            fs = 0.58 * fy *(Ast_x_positive/(np.ceil(Ast_x_positive/((np.pi)*((diameter_rod/2) **2 ))) * ((np.pi)*((diameter_rod/2) **2 ))))
            
            if((length_ex/1000) > 10):
                beta =  10/(length_ex/1000)
            else:
                beta = 1
            gamma = min(1/(0.225 + (0.0032 *fs) + (0.625/2.303)*np.log(Ast_shear_function(spacing_x_midspan)[1])),2)
            
            if((length_ex/depth_concrete_initial) > 26*beta*gamma):
                print(" sahi data lo na baby ji")
        
            development_length = (diameter_rod * 0.87*fy)/(4*1.6*clause26_2_1_1.iloc[0,int((fck/5)-3)])
            
            
            
            #provide distribution bar
            Ast_dist = Ast_minimum
            spacing_dist = min((1000 * Ast_individual)/(Ast_dist), 450, 5*depth_concrete_initial)
            if(spacing_dist !=  (1000 * Ast_individual)/(Ast_dist)):
                Ast_dist = (1000 * Ast_individual)/(spacing_dist)
            
            
            #Torsion bar
            length_torsion_bar = length_ex/5
            spacing_torsional_bar = (4*Ast_x_positive)/3
            bars_required = np.ceil((length_torsion_bar/spacing_torsional_bar) +1)
            
            
            #crank distribution
            crank_bars_num_x = np.ceil((Ast_x_negative - Ast_x_positive)/(np.pi * (1/4) * (diameter_rod**2)))
            spacing_crank_bar_x = length_ex/crank_bars_num_x
        
            crank_bars_num_y = np.ceil((Ast_y_negative - Ast_y_positive)/(np.pi * (1/4) * (diameter_rod**2)))
            spacing_crank_bar_y = length_ex/crank_bars_num_y  
            
            #printing values
            if(times_depth_change>0):
                print(f"\n\n depth without mortar and rod after correcting to hold the moment {depth_concrete_initial}\n\n")
            
            if(times_depth_change > 7):
                print("the data you entered is giving less depth of slab which in returns does not hold moment. so, change data according")
            
            
            if(temp_index_midspan > 7):
                print("the spacing along midspan we are getting from your data is not worthy of designining slab. so, change your data")
            
            if(temp_index_spacing > 7):
                print("the spacing along endspan we are getting from your data is not worthy of designining slab. so, change your data")
            
            
            if((times_depth_change <= 7) and (temp_index_midspan <= 7) and (temp_index_spacing<=7)):
                print(f"Depth we get after taking rod and mortar in consideration is {depth_concrete} mm")
                print(f"we need {crank_bars_num_x} crank bars at distance {spacing_crank_bar_x} along short side")
                print(f"spacing of basic torsion bars along short side:{round(spacing_x_midspan,3)} mm")
                print(f"we need {crank_bars_num_y} crank bars at distance {spacing_crank_bar_y} along long side")
                print(f"spacing of basic torsion bars along long side:{round(spacing_y_midspan,3)} mm")
                print(f"Provided that slab is not continuous from sides, {bars_required * (breadth_room/1000)} bars of {diameter_rod}mm at {spacing_torsional_bar}mm for given room")
                print(f"We take {diameter_rod}mm rod at space {spacing_dist}mm")
                print(f"Development_length is {round(development_length,3)}")
                
        
        
        else:
            print("Oops!!!, Sorry, we can't calculate for the dimensons you have given")
            
    

main(depth_concrete_initial)

Depth we get after taking rod and mortar in consideration is 145.0 mm
we need 1.0 crank bars at distance 4125.0 along short side
spacing of basic torsion bars along short side:375.0 mm
we need 1.0 crank bars at distance 4125.0 along long side
spacing of basic torsion bars along long side:375.0 mm
Provided that slab is not continuous from sides, 25.0 bars of 10.0mm at 255.68656411722364mm for given room
We take 10.0mm rod at space 450mm
Development_length is 470.117
8379089.296874998
